In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np
import pandas_profiling as pp
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from mlxtend.plotting import scatterplotmatrix
from sklearn.feature_selection import mutual_info_regression
from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_squared_error

#models
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor
from mlxtend.regressor import StackingCVRegressor

In [ ]:
dataInput ='/kaggle/input/melbourne-housing-snapshot/melb_data.csv'

In [ ]:
data = pd.read_csv(dataInput)


# Exploratory Data Analysis

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
missing_values_count = data.isnull().sum()

In [ ]:
missing_values_count

In [ ]:
cols_with_unique = [len(data[col].unique()) for col in data.columns]

cols_with_unique

In [ ]:
report = pp.ProfileReport(data)
report

# Feature Engineering

## Fill missing values

In [ ]:
#If a value is missing becuase it doesn't exist,I would like to replace the missing values with 0
data['Car'] = data['Car'].fillna(0)

## Drop columns

In [ ]:
#Drop 'Address' since its high cardinality
cols_to_be_removed = ["Address"]
data.drop(cols_to_be_removed,inplace=True,axis = 1)

In [ ]:
#object columns
features_nom = ['Suburb','Type','Method','SellerG','Date','Regionname','CouncilArea']

def to_category(df):
    for name in features_nom:
            df[name] = df[name].astype("category")
    return df

In [ ]:
data = to_category(data)

In [ ]:
data.info()

## Label encoding for categoricals

In [ ]:
# Label encoding is good for XGBoost and RandomForest, but one-hot
# would be better for models like Lasso or Ridge.
for colname in data.select_dtypes(["category"]):
        data[colname] = data[colname].cat.codes

## Deep dive features

In [ ]:
# Let's visualize the features in the dataset
scatterplotmatrix(data.values,figsize= (120,100),names = data.columns, alpha=0.5)

In [ ]:
# The heatmap below shows that the Melbourne Heatmap of the House Prices，
fig = px.density_mapbox(data, lat='Lattitude', lon='Longtitude', z='Price', radius=10,
                        center=dict(lat=-37.8, lon=145), zoom=10,
                        mapbox_style="stamen-terrain", opacity = 0.5, title = 'Melbourne Price Heatmap')
fig.show()

In [ ]:
# visualising some more outliers in the data values
cols = ['Price','Bedroom2','Landsize','BuildingArea','YearBuilt']
scatterplotmatrix(data[cols].values,figsize= (60,50),names = cols, alpha=0.5)


## Remove Outliers

In [ ]:
data.drop(data[(data['Bedroom2'] > 12)].index, inplace = True)
data.drop(data[(data['Landsize'] > 30000)].index, inplace = True)
data.drop(data[(data['BuildingArea'] > 1200)].index, inplace = True)
data.drop(data[(data['YearBuilt'] < 1600)].index, inplace = True)

## Impute the missing values

In [ ]:
# Although the BuildingArea, YearBuilt features have pretty high missing values,these features
# ended up helping performance and I replace the missing values with mean
data['BuildingArea'].fillna(data['BuildingArea'].mean(), inplace = True)

In [ ]:
data['YearBuilt'].fillna(data['YearBuilt'].mean(), inplace = True)

In [ ]:
data.info()

In [ ]:
# Reset index since remove outliers before
data.reset_index(drop=True,inplace= True)

## Create features

In [ ]:
#interactions
newLandsize = pd.get_dummies(data.Type, prefix="TypeL")
newLandsize = newLandsize.mul(data.Landsize, axis=0)#


In [ ]:
#group_transforms
groupX = pd.DataFrame()
groupX["PostcodeMeanDistance"] = data.groupby("Postcode")["Distance"].transform("mean")


## Create features by clustering

In [ ]:

cluster_features = [
    "Lattitude",
    "Longtitude",
    "Distance",
    "Landsize"
]

def cluster_labels(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=123)
    X_new = pd.DataFrame()
    X_new["Cluster"] = kmeans.fit_predict(X_scaled)
    return X_new


def cluster_distance(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=20, n_init=50, random_state=123)
    X_cd = kmeans.fit_transform(X_scaled)
    # Label features and join to dataset
    X_cd = pd.DataFrame(
        X_cd, columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])]
    )
    return X_cd

In [ ]:
temp = cluster_labels(data, cluster_features, n_clusters=20)

## Create features by PCA

In [ ]:

pca_features = [
    "Rooms",
    "Type",
    "Landsize",
    "Distance",
    "Lattitude",
    "Longtitude"    
]

def apply_pca(X, standardize=True):
    # Standardize
    if standardize:
        X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Create principal components
    pca = PCA()
    X_pca = pca.fit_transform(X)
    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names)
    # Create loadings
    loadings = pd.DataFrame(
        pca.components_.T,  # transpose the matrix of loadings
        columns=component_names,  # so the columns are the principal components
        index=X.columns,  # and the rows are the original features
    )
    return pca, X_pca, loadings

def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

In [ ]:
pca, X_pca, loadings = apply_pca(data[pca_features])
plot_variance(pca)

In [ ]:
loadings

In [ ]:
X_pca.head()

In [ ]:
data = data.join(newLandsize)
data = data.join(groupX)

# These features ended up not helping performance
# data = data.join(temp)
# data = data.join(X_pca["PC1"])


## Create features by Target Encoding

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train,test = train_test_split(data,random_state = 1,test_size = 0.2)

In [ ]:
class CrossFoldEncoder:
    def __init__(self, encoder, **kwargs):
        self.encoder_ = encoder
        self.kwargs_ = kwargs  # keyword arguments for the encoder
        self.cv_ = KFold(n_splits=5)

    # Fit an encoder on one split and transform the feature on the
    # other. Iterating over the splits in all folds gives a complete
    # transformation. We also now have one trained encoder on each
    # fold.
    def fit_transform(self, X, y, cols):
        self.fitted_encoders_ = []
        self.cols_ = cols
        X_encoded = []
        for idx_encode, idx_train in self.cv_.split(X):
            fitted_encoder = self.encoder_(cols=cols, **self.kwargs_)
            fitted_encoder.fit(
                X.iloc[idx_encode, :], y.iloc[idx_encode],
            )
            X_encoded.append(fitted_encoder.transform(X.iloc[idx_train, :])[cols])
            self.fitted_encoders_.append(fitted_encoder)
        
        X_encoded = pd.concat(X_encoded)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded
# To transform the test data, average the encodings learned from
    # each fold.
    def transform(self, X):
        from functools import reduce

        X_encoded_list = []
        for fitted_encoder in self.fitted_encoders_:
            X_encoded = fitted_encoder.transform(X)
            X_encoded_list.append(X_encoded[self.cols_])
        X_encoded = reduce(
            lambda x, y: x.add(y, fill_value=0), X_encoded_list
        ) / len(X_encoded_list)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

In [ ]:
X = train.copy()
y = X.pop("Price")

In [ ]:
encoder = CrossFoldEncoder(MEstimateEncoder, m=1)
# This feature ended up not helping performance
# X = X.join(encoder.fit_transform(X, y, cols=["Method"]))

## Cross validation

In [ ]:
# RMSLE (Root Mean Squared Log Error)
def score_dataset(X, y, model=XGBRegressor()):    

    log_y = np.log(y)
    score = cross_val_score(
        model, X, log_y, cv=5, scoring="neg_mean_squared_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
     
    return score

In [ ]:
baseline_score = score_dataset(X, y)
print(f"Baseline score: {baseline_score:.5f} RMSLE")

## Mutual Information

* **Locate features with the most potential**

In [ ]:
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    print(discrete_features)
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=123)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [ ]:
X = train.copy()
y = X.pop("Price")

mi_scores = make_mi_scores(X, y)
mi_scores

In [ ]:
plot_mi_scores(mi_scores[mi_scores > 0.0])

## Permutation Importance

* **What features does model think are important**

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=123)
my_model = XGBRegressor().fit(train_X, train_y)

perm = PermutationImportance(my_model, random_state=123).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist(),top = 50)

# Models and Evaluation Metrics

In [ ]:

X = train.copy()
y = X.pop("Price")

In [ ]:
X.info()

In [ ]:
X.head()

## Setup models

In [ ]:
# Hyperparameter Tuning

# import optuna

# def objective(trial):
#     xgb_params = dict(
#         max_depth=trial.suggest_int("max_depth", 2, 10),
#         learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-2, log=True),
#         n_estimators=trial.suggest_int("n_estimators", 1000, 8000),
#         min_child_weight=trial.suggest_int("min_child_weight", 1, 10),
#         colsample_bytree=trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         subsample=trial.suggest_float("subsample", 0.2, 1.0),
#         reg_alpha=trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True),
#         reg_lambda=trial.suggest_float("reg_lambda", 1e-2, 1e-1, log=True),
#     )
#     xgb = XGBRegressor(**xgb_params)
#     return score_dataset(X, y, xgb)

# study = optuna.create_study(direction="minimize")
# study.optimize(objective, n_trials=20)
# xgb_params = study.best_params

In [ ]:
catboost = CatBoostRegressor(learning_rate = 0.067, 
                             iterations = 7900, 
                             depth = 6, 
                             l2_leaf_reg = 12,
                             verbose = False
                            )

lightgbm = LGBMRegressor(objective='regression', 
                           num_leaves=29,
                           learning_rate=0.005191679498442796, 
                           n_estimators=7475,
                           max_bin=255, 
                           bagging_fraction=0.6241432209263227,
                           bagging_freq=3, 
                           bagging_seed=8,
                           feature_fraction=0.4929878468515065,
                           feature_fraction_seed=8,
                           min_sum_hessian_in_leaf = 9.285861648865987,
                           verbose= -1,
                           random_state=123)

xgboost = XGBRegressor( max_depth= 9, 
                        learning_rate= 0.008, 
                        n_estimators=2834, 
                        min_child_weight= 7, 
                        colsample_bytree= 0.2875, 
                        subsample= 0.85, 
                        reg_alpha= 0.00032, 
                        reg_lambda= 0.0144,
                        random_state=123)

rf = RandomForestRegressor(n_estimators=2000,
                           random_state=123)

gbr = GradientBoostingRegressor(n_estimators=7300,
                                learning_rate=0.015,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=36,
                                min_samples_split=45,
                                loss='huber',
                                random_state=123)  


# Stack up all the models above, optimized using xgboost
stack_gen = StackingCVRegressor(regressors=(xgboost, lightgbm, catboost, gbr, rf),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

## Train models

### Get cross validation scores for each model

In [ ]:
scores = {}

score = score_dataset(X, y, catboost)
print("catboost: {:.4f} ".format(score))
scores['catboost'] = score


In [ ]:

score = score_dataset(X, y, lightgbm)
print("lightgbm: {:.4f} ".format(score))
scores['lightgbm'] = score


In [ ]:

score = score_dataset(X, y, xgboost)
print("xgboost: {:.4f} ".format(score))
scores['xgboost'] = score


In [ ]:
score = score_dataset(X, y, rf)
print("rf: {:.4f} ".format(score))
scores['rf'] = score

In [ ]:
score = score_dataset(X, y, gbr)
print("gbr: {:.4f} ".format(score))
scores['gbr'] = score

### Fit the models

In [ ]:
print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X), np.array(np.log(y)))


In [ ]:
print('catboost')
catb_model_full_data = catboost.fit(X, np.log(y))

In [ ]:
print('lightgbm')
lgb_model_full_data = lightgbm.fit(X, np.log(y))

In [ ]:
print('xgboost')
xgb_model_full_data = xgboost.fit(X, np.log(y))

In [ ]:
print('RandomForest')
rf_model_full_data = rf.fit(X, np.log(y))

In [ ]:
print('GradientBoosting')
gbr_model_full_data = gbr.fit(X, np.log(y))

## Blend models and get predictions

In [ ]:
# Blend models in order to make the final predictions more robust to overfitting
def blended_predictions(X):
    return ((0.15 * catb_model_full_data.predict(X)) +        
            (0.05 * gbr_model_full_data.predict(X)) +
            (0.15 * xgb_model_full_data.predict(X)) + 
            (0.15 * lgb_model_full_data.predict(X)) +
            (0.5 * stack_gen_model.predict(np.array(X)))
           )

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
blended_score = rmsle(np.log(y), blended_predictions(X))
scores['blended'] = blended_score
print('RMSLE score on train data:')
print(blended_score)

# Evaluate on Test Data

In [ ]:
test_X = test.copy()
test_y = test_X.pop("Price")

In [ ]:
TestScores = {}

In [ ]:
score = rmsle(np.log(test_y), catb_model_full_data.predict(test_X))
print('CatBoost RMSLE score on test data: {:.4f}'.format(score))
TestScores['catboost'] = score

In [ ]:
score = rmsle(np.log(test_y), lgb_model_full_data.predict(test_X))
print('LGBM RMSLE score on test data: {:.4f}'.format(score))
TestScores['lightgbm'] = score

In [ ]:
score = rmsle(np.log(test_y), xgb_model_full_data.predict(test_X))
print('XGB RMSLE score on test data: {:.4f}'.format(score))
TestScores['xgboost'] = score

In [ ]:
score = rmsle(np.log(test_y), rf_model_full_data.predict(test_X))
print('RandomForest RMSLE score on test data: {:.4f}'.format(score))
TestScores['rf'] = score

In [ ]:
score = rmsle(np.log(test_y), gbr_model_full_data.predict(test_X))
print('GradientBoosting RMSLE score on test data: {:.4f}'.format(score))
TestScores['gbr'] = score

In [ ]:
score = rmsle(np.log(test_y), stack_gen_model.predict(np.array(test_X)))
print('Stack RMSLE score on test data: {:.4f}'.format(score))
# TestScores['stack'] = score

In [ ]:
blended_score = rmsle(np.log(test_y), blended_predictions(test_X))
print('Blended RMSLE score on test data: {:.4f}'.format(blended_score))
TestScores['blended'] = blended_score

In [ ]:
import matplotlib.patches as mpatches

green_patch = mpatches.Patch(color='green', label='Test Data')
blue_patch = mpatches.Patch(color='blue', label='Train Data')

# Plot the predictions for each model
sns.set_style("white")
fig = plt.figure(figsize=(24, 12))

ax1 = sns.pointplot(x=list(scores.keys()), y=list(scores.values()), markers=['s'], linestyles=['-'],color='blue')
for i, score in enumerate(scores.values()):
    ax1.text(i, score, '{:.4f}'.format(score), horizontalalignment='left', size='large', color='black', weight='semibold')

ax2 = sns.pointplot(x=list(TestScores.keys()), y=list(TestScores.values()), markers=['o'], linestyles=['-'],color='green')
for i, score in enumerate(TestScores.values()):
    ax2.text(i, score, '{:.4f}'.format(score), horizontalalignment='left', size='large', color='black', weight='semibold')

plt.ylabel('Score (RMSLE)', size=20, labelpad=12.5)
plt.xlabel('Model', size=20, labelpad=12.5)
plt.tick_params(axis='x', labelsize=13.5)
plt.tick_params(axis='y', labelsize=12.5)

plt.title('Scores of Models', size=20)
plt.legend(handles=[green_patch, blue_patch])

plt.show()

#### Reference:
[https://www.kaggle.com/lavanyashukla01/how-i-made-top-0-3-on-a-kaggle-competition](http://)

[https://www.kaggle.com/learn/feature-engineering](http://)